In [2]:
import h5py
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

In [3]:
f = h5py.File('datasets/Blip_chunk17_H1_O3a_new.hdf5', 'r')
#f2 = pandas.read_hdf('datasets/Blip_chunk17_H1_O3a_new.hdf5', key = 'snr', mode= 'r')

print(list(f.keys())) 

blipdata = f['snr']
print(blipdata.shape)
"""
b = h5py.File('datasets/inj_17.hdf5', 'r')
print(list(b.keys()))
injdata = b['H1']
print(injdata.shape)
"""
t = h5py.File('datasets/template_bank.hdf5', 'r')
#print(list(t.keys()))
#tempdata = t['1977861.0']
#print(np.array(tempdata))


['Gamma0', 'chisq', 'end_time', 'end_time_ns', 'glitch_end_time', 'glitch_end_time_decimal', 'glitchnum', 'ifo', 'snr']
(16648,)


In [17]:

blip_csv1 = pd.read_csv('/data/gravwav/lopezm/ML_projects/Projects_2022/Project_3/Data/dataframes/Blip_H1_O3a.csv')
blip_np = blip_csv1.to_numpy()

blip_sorted = np.array(blip_csv1.sort_values(by = ['Event ID', 'Event time']))
event_amount = len(blip_sorted)

ev_snr = np.array([])
av_snr = np.array([])
ev_chisq = np.array([])
av_chisq = np.array([])
ev_m1 = np.array([])
av_m1 = np.array([])
ev_m2 = np.array([])
av_m2 = np.array([])
ev_s1 = np.array([])
av_s1 = np.array([])
ev_s2 = np.array([])
av_s2 = np.array([])
prev_evtime = blip_sorted[0][4]

for i in range(event_amount):
    event_id = int(blip_sorted[i][3])
    
    current_evtime = blip_sorted[i][4]
    if current_evtime == prev_evtime:
        ev_snr = np.append(ev_snr, blip_sorted[i][5])
        ev_chisq = np.append(ev_chisq, blip_sorted[i][6])
        ev_m1 = np.append(ev_m1, blip_sorted[i][7])
        ev_m2 = np.append(ev_m2, blip_sorted[i][8])
        ev_s1 = np.append(ev_s1, blip_sorted[i][9])
        ev_s2 = np.append(ev_s2, blip_sorted[i][10])
        
        #print('Appending event' ,current_evtime, 'm1', ev_m1)
    else: 
        if len(ev_snr) ==0:
            av_snr = np.append(av_snr, blip_sorted[i][5])
            av_chisq = np.append(av_chisq, blip_sorted[i][6])
            av_m1 = np.append(av_m1, blip_sorted[i][7])
            av_m2 = np.append(av_m2, blip_sorted[i][8])
            av_s1 = np.append(av_s1, blip_sorted[i][9])
            av_s2 = np.append(av_s2, blip_sorted[i][10])
        else:
            av_snr = np.append(av_snr, np.average(ev_snr))
            av_chisq = np.append(av_chisq, np.average(ev_chisq))
            av_m1 = np.append(av_m1, np.average(ev_m1))
            av_m2 = np.append(av_m2, np.average(ev_m2))
            av_s1 = np.append(av_s1, np.average(ev_s1))
            av_s2 = np.append(av_s2, np.average(ev_s2))    
        
        ev_snr = np.array([])
        ev_chisq = np.array([])
        ev_m1 = np.array([])
        ev_m2 = np.array([])
        ev_s1 = np.array([])
        ev_s2 = np.array([])
        
        #print('Event time', prev_evtime, 'SNR', av_snr, 'Chisq', av_chisq, 'M1', av_m1, 'M2', av_m2, 'S1', av_s1, 'S2', av_s2)
        prev_evtime = current_evtime


triggers = np.array((av_snr, av_chisq, av_m1, av_m2, av_s1, av_s2))


SNR (994,)
(5964,)


In [25]:
class Blip_dataset(Dataset):
    def __init__(self, path:str ):
        self.path = path
        blip_csv1 = pd.read_csv(path)
        blip_np = blip_csv1.to_numpy()
        
        self.triggers = np.array(blip_csv1.sort_values(by = ['Event ID', 'Event time']))
        self.trigger_amount = len(self.triggers)
    def __len__(self):
        return self.trigger_amount
    
    def load_set(self):
        
        ev_snr = np.array([])
        av_snr = np.array([])
        ev_chisq = np.array([])
        av_chisq = np.array([])
        ev_m1 = np.array([])
        av_m1 = np.array([])
        ev_m2 = np.array([])
        av_m2 = np.array([])
        ev_s1 = np.array([])
        av_s1 = np.array([])
        ev_s2 = np.array([])
        av_s2 = np.array([])
        prev_evtime = self.triggers[0][4]

        for i in range(event_amount):
            event_id = int(self.triggers[i][3])

            current_evtime = self.triggers[i][4]
            if current_evtime == prev_evtime:
                ev_snr = np.append(ev_snr, self.triggers[i][5])
                ev_chisq = np.append(ev_chisq, self.triggers[i][6])
                ev_m1 = np.append(ev_m1, self.triggers[i][7])
                ev_m2 = np.append(ev_m2, self.triggers[i][8])
                ev_s1 = np.append(ev_s1, self.triggers[i][9])
                ev_s2 = np.append(ev_s2, self.triggers[i][10])

                #print('Appending event' ,current_evtime, 'm1', ev_m1)
            else: 
                if len(ev_snr) ==0:
                    av_snr = np.append(av_snr, self.triggers[i][5])
                    av_chisq = np.append(av_chisq, self.triggers[i][6])
                    av_m1 = np.append(av_m1, self.triggers[i][7])
                    av_m2 = np.append(av_m2, self.triggers[i][8])
                    av_s1 = np.append(av_s1, self.triggers[i][9])
                    av_s2 = np.append(av_s2, self.triggers[i][10])
                else:
                    av_snr = np.append(av_snr, np.average(ev_snr))
                    av_chisq = np.append(av_chisq, np.average(ev_chisq))
                    av_m1 = np.append(av_m1, np.average(ev_m1))
                    av_m2 = np.append(av_m2, np.average(ev_m2))
                    av_s1 = np.append(av_s1, np.average(ev_s1))
                    av_s2 = np.append(av_s2, np.average(ev_s2))    

                ev_snr = np.array([])
                ev_chisq = np.array([])
                ev_m1 = np.array([])
                ev_m2 = np.array([])
                ev_s1 = np.array([])
                ev_s2 = np.array([])
                
                prev_evtime = current_evtime
        trigger_average = np.array((av_snr, av_chisq, av_m1, av_m2, av_s1, av_s2))
        return trigger_average
    def __getitem__(self, idx): 

        trigger_average = self.load_set()
        return trigger_average



In [26]:
blip_dataset = Blip_dataset('/data/gravwav/lopezm/ML_projects/Projects_2022/Project_3/Data/dataframes/Blip_H1_O3a.csv')
triggers = blip_dataset[0]

print(triggers)

[[ 1.21973650e+01  1.11598543e+01  7.68060680e+00 ...  8.52358440e+00
   8.30062390e+00  8.31931500e+00]
 [ 3.83635720e+00  7.08697295e+00  1.95594270e+00 ...  1.98094840e+00
   1.63696540e+00  1.68528220e+00]
 [ 4.82654887e+01  4.89758490e+01  4.48161280e+01 ...  5.03115880e+01
   1.51303830e+02  5.90329210e+01]
 [ 5.26231050e+00  5.37316585e+00  5.44613500e+00 ...  1.97927060e+01
   1.94313510e+01  8.61657330e+00]
 [ 3.73722553e-01 -5.14684390e-01  9.79754210e-01 ...  3.64021600e-01
  -6.50441470e-01  4.65813340e-01]
 [-3.61054267e-01 -5.90630245e-01 -8.69801700e-01 ... -9.49819560e-01
   1.10021030e-01 -9.27850900e-01]]
